In [423]:
import sys
sys.path.insert(0, '/Users/nathanieldake/.virtualenvs/intuitiveml-venv/lib/python3.7/site-packages')

import requests
import io 

import pandas as pd
import numpy as np
import Levenshtein

# 1. Cases by county data
Found: https://github.com/nytimes/covid-19-data

In [424]:
url = 'https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv'
r = requests.get(url)

In [425]:
df = pd.read_csv(io.BytesIO(r.content))

In [426]:
df.sample(5)

,date,county,state,fips,cases,deaths
565,2020-03-05,New York City,New York,NaN,4,0
6585,2020-03-21,Grenada,Mississippi,28043.0,1,0
1441,2020-03-12,Honolulu,Hawaii,15003.0,2,0
14383,2020-03-27,Meriwether,Georgia,13199.0,3,0
40793,2020-04-08,Petersburg Borough,Alaska,2195.0,1,0


In [427]:
df.shape

(45881, 6)

# 2. Zip code map

In [428]:
df_zip_map = pd.read_csv('zip_code_database.csv')

In [429]:
df_zip_map.head()

,zip,type,primary_city,acceptable_cities,unacceptable_cities,state,county,timezone,area_codes,latitude,longitude,world_region,country,decommissioned,estimated_population,notes
0,501,UNIQUE,Holtsville,NaN,I R S Service Center,NY,Suffolk County,America/New_York,631,40.81,-73.04,NaN,US,0,384,NaN
1,544,UNIQUE,Holtsville,NaN,Irs Service Center,NY,Suffolk County,America/New_York,631,40.81,-73.04,NaN,US,0,0,NaN
2,601,STANDARD,Adjuntas,NaN,"Colinas Del Gigante, Jard De Adjuntas, Urb San...",PR,Adjuntas,America/Puerto_Rico,"787,939",18.16,-66.72,NaN,US,0,0,NaN
3,602,STANDARD,Aguada,NaN,"Alts De Aguada, Bo Guaniquilla, Comunidad Las ...",PR,NaN,NaN,787,18.38,-67.18,NaN,US,0,0,NaN
4,603,STANDARD,Aguadilla,Ramey,"Bda Caban, Bda Esteves, Bo Borinquen, Bo Ceiba...",PR,Aguadilla,America/Puerto_Rico,787,18.43,-67.15,NaN,US,0,0,NaN


# 3. Expansion dataset join class
Problem we are trying to solve:
* How do we map `New York County` to `New York City`?
* We can perform a simple `.lower()` and drop `County` from all county names, but even then we must be able to map: `new york` to `new york city`. 
* This will clearly be an issue moving forward. Consider, another example, mapping `new york city` to `new york, city`. If we have 100,000 rows it will be nearly impossible to realize there is a random comma that we need to get rid of. No preprocessing will be perfect and allow for us to always have perfect joins.

In [142]:
df.head()

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0
1,2020-01-22,Snohomish,Washington,53061.0,1,0
2,2020-01-23,Snohomish,Washington,53061.0,1,0
3,2020-01-24,Cook,Illinois,17031.0,1,0
4,2020-01-24,Snohomish,Washington,53061.0,1,0


In [143]:
df_zip_map[['zip', 'county']].sample(5)

,zip,county
34259,78101,Bexar County
18042,41528,Pike County
26123,59644,Broadwater County
23132,53020,Sheboygan County
17681,40384,Woodford County


### 3.1 Define a preprocess function for each join col 

In [8]:
def preprocess_county(county_col):
    """Lower and remove County from end"""
    return county_col.str.lower().str.replace(' county', '')

In [12]:
a = preprocess_county(df.county)
b = preprocess_county(df_zip_map.county)
a.isin(b).sum()

39284

In [9]:
df = df.assign(
    **{f'processed_county': preprocess_county(df.county)}
)

In [10]:
df_zip_map = df_zip_map.assign(
    **{f'processed_county': preprocess_county(df_zip_map.county)}
)

### 3.2 Based on preprocessed join col, find closest match (via apply)
* 1000 takes 23 seconds
* 2000 takes 47 seconds
* 3000 takes 70 seconds

* After only passing in unique values, down to 6.25 seconds!
* Full set takes 1 min 18 seconds
* Full set takes 3 seconds once calling unique twice!

In [15]:
def closest_match(row, col_map=None):
    sorted_dist_indices = col_map.apply(lambda x: Levenshtein.distance(x, row)).sort_values().index
    return col_map[sorted_dist_indices].iloc[0]

In [17]:
%%time
output = pd.Series(df.processed_county.dropna()).apply(
    closest_match, col_map=pd.Series(df_zip_map.processed_county.unique()).dropna()
)

CPU times: user 1min 27s, sys: 310 ms, total: 1min 28s
Wall time: 1min 28s


### 3.3 Based on preprocessed join col, find closest match (via for loop)

In [20]:
def closest_match(str_1, str_2):
    return Levenshtein.distance(str_1, str_2)

closest_match_vectorized = np.vectorize(closest_match)

In [22]:
def create_str_map(col1, col2):
    str_map = {}
    str_array = col2.dropna().unique()
    for x in col1.dropna().unique():
        distances = closest_match_vectorized(x, str_array)
        min_idx = np.argmin(distances)
        str_map[x] = str_array[min_idx]
    return str_map

In [23]:
%%time
str_map = create_str_map(df.processed_county, df_zip_map.processed_county)

CPU times: user 1.44 s, sys: 5.84 ms, total: 1.45 s
Wall time: 1.45 s


###  3.4 Apply map

In [105]:
df.head()

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0
1,2020-01-22,Snohomish,Washington,53061.0,1,0
2,2020-01-23,Snohomish,Washington,53061.0,1,0
3,2020-01-24,Cook,Illinois,17031.0,1,0
4,2020-01-24,Snohomish,Washington,53061.0,1,0


In [24]:
df = df.assign(
    join_county=df.processed_county.map(str_map)
)
df.head()

,date,county,state,fips,cases,deaths,processed_county,join_county
0,2020-01-21,Snohomish,Washington,53061.0,1,0,snohomish,snohomish
1,2020-01-22,Snohomish,Washington,53061.0,1,0,snohomish,snohomish
2,2020-01-23,Snohomish,Washington,53061.0,1,0,snohomish,snohomish
3,2020-01-24,Cook,Illinois,17031.0,1,0,cook,cook
4,2020-01-24,Snohomish,Washington,53061.0,1,0,snohomish,snohomish


### 3.5 Join 

In [25]:
df_final = pd.merge(
    df,
    df_zip_map[['zip', 'processed_county']],
    how='left',
    left_on='join_county',
    right_on='processed_county'
)
print(df_final.shape)

(2973707, 10)


---
# 4. Implement Parallelization
### 4.1 Parallelize Finding Closest Match - Basic Python

In [67]:
counties = df.processed_county
counties_map = df_zip_map.processed_county

In [14]:
import multiprocessing
from itertools import repeat


def create_str_map_chunked(col1_chunk, col2):
    str_map = {}
    str_array = col2.dropna().unique()
    for x in col1_chunk.dropna().unique():
        distances = closest_match_vectorized(x, str_array)
        min_idx = np.argmin(distances)
        str_map[x] = str_array[min_idx]
    return str_map

nproc = multiprocessing.cpu_count()

data_split = np.array_split(counties, nproc)



In [15]:

%%time
with multiprocessing.Pool(multiprocessing.cpu_count()) as pool:
    res = pool.starmap(create_str_map_chunked, zip(data_split, repeat(counties_map)))

CPU times: user 114 ms, sys: 54.1 ms, total: 168 ms
Wall time: 2.42 s


In [72]:
str_map = {k: v for d in res for k, v in d.items()}

### 4.2 Parallelize Finding Closest Match - Dask Delayed

In [12]:
import pandas as pd
import dask.dataframe as dd
from dask.multiprocessing import get
from dask import delayed

In [154]:
@delayed
def create_str_map_dask(col1_chunk, col2):
    str_map = {}
    str_array = col2.dropna().unique()
    for x in col1_chunk.dropna().unique():
        distances = closest_match_vectorized(x, str_array)
        min_idx = np.argmin(distances)
        str_map[x] = str_array[min_idx]
    return str_map

In [155]:
results = []
for col1_chunk in np.array_split(counties, nproc):
    results.append(create_str_map_dask(col1_chunk, counties_map))

In [156]:
%%time
output = delayed(results).compute()

CPU times: user 13.3 s, sys: 190 ms, total: 13.5 s
Wall time: 13.3 s


In [157]:
str_map = {k: v for d in output for k, v in d.items()}

In [158]:
len(str_map)

1548

In [167]:
str_map['cook']

'cook'

In [103]:
str_map['new york city']

'new york'

### 4.3 Parallelize Finding Closest Match - Dask Map Partitions

In [16]:
ddf = dd.from_pandas(df, npartitions=nproc)

In [40]:
def create_str_map_dask_map_partitions(df, col1, df2=None, col2=None):
    str_map = {}
    
    df_col_arr = df[col1].dropna().unique()
    df2_col_arr = df2[col2].dropna().unique()
    
    for x in df_col_arr:
        distances = closest_match_vectorized(x, df2_col_arr)
        min_idx = np.argmin(distances)
        str_map[x] = df2_col_arr[min_idx]
        
    return str_map

In [41]:
df_zip_map['processed_county'].dropna().unique()

array(['suffolk', 'adjuntas', 'aguadilla', ...,
       'ketchikan gateway borough',
       'prince of wales-outer ketchikan borough', 'wrangell borough'],
      dtype=object)

In [42]:
%%time
output = ddf.map_partitions(
    create_str_map_dask_map_partitions, 
    'processed_county', 
    df2=df_zip_map,
    col2='processed_county'
).compute()

CPU times: user 12.9 s, sys: 184 ms, total: 13.1 s
Wall time: 13 s


In [43]:
str_map_2 = {k: v for d in output for k, v in d.items()}

In [44]:
len(str_map_2)

1548

In [49]:
str_map_2['cook']

'cook'

# 6. All in Dask

In [51]:
ddf = dd.from_pandas(df, npartitions=12) # TODO: Don't hard code
ddf_zip_map = dd.from_pandas(df_zip_map, npartitions=12)

In [52]:
ddf = ddf.assign(
    **{f'processed_county': preprocess_county(ddf.county)}
)
ddf_zip_map = ddf_zip_map.assign(
    **{f'processed_county': preprocess_county(ddf_zip_map.county)}
)

In [53]:
def create_str_map_dask_map_partitions(df, col1, df2=None, col2=None):
    str_map = {}
    
    df_col_arr = df[col1].dropna().unique()
    df2_col_arr = df2[col2].dropna().unique()
    
    for x in df_col_arr:
        distances = closest_match_vectorized(x, df2_col_arr)
        min_idx = np.argmin(distances)
        str_map[x] = df2_col_arr[min_idx]
        
    return str_map

In [54]:
col2 = ddf_zip_map.processed_county.compute()

In [55]:
%%time
output = ddf.map_partitions(
    create_str_map_dask_map_partitions, 
    'processed_county', 
    df2=df_zip_map,
    col2='processed_county'
).compute()

CPU times: user 13.5 s, sys: 202 ms, total: 13.7 s
Wall time: 13.6 s


In [56]:
str_map_3 = {k: v for d in output for k, v in d.items()}

In [59]:
str_map_3['cook']

'cook'

### Just on series

In [79]:
def create_str_map_dask_series_map_partitions(col1, col2=None):
    str_map = {}
    
    col1_arr = col1.dropna().unique()
    col2_arr = col2.dropna().unique()
    
    for x in col1_arr:
        distances = closest_match_vectorized(x, col2_arr)
        min_idx = np.argmin(distances)
        str_map[x] = col2_arr[min_idx]
        
    return str_map

In [80]:
%%time
output = ddf.processed_county.map_partitions(
    create_str_map_dask_series_map_partitions,
    col2=ddf_zip_map.processed_county.compute()
).compute()

CPU times: user 13.1 s, sys: 181 ms, total: 13.3 s
Wall time: 13.2 s


In [81]:
str_map_4 = {k: v for d in output for k, v in d.items()}

In [82]:
str_map_4

{'snohomish': 'snohomish',
 'cook': 'cook',
 'orange': 'orange',
 'maricopa': 'maricopa',
 'los angeles': 'los angeles',
 'santa clara': 'santa clara',
 'suffolk': 'suffolk',
 'san francisco': 'san francisco',
 'dane': 'dane',
 'san diego': 'san diego',
 'bexar': 'bexar',
 'douglas': 'douglas',
 'humboldt': 'humboldt',
 'sacramento': 'sacramento',
 'solano': 'solano',
 'spokane': 'spokane',
 'salt lake': 'salt lake',
 'marin': 'marin',
 'napa': 'napa',
 'sonoma': 'sonoma',
 'washington': 'washington',
 'king': 'king',
 'alameda': 'alameda',
 'hillsborough': 'hillsborough',
 'manatee': 'manatee',
 'new york city': 'new york',
 'unknown': 'union',
 'placer': 'placer',
 'san mateo': 'san mateo',
 'fulton': 'fulton',
 'norfolk': 'norfolk',
 'grafton': 'grafton',
 'walla walla': 'walla walla',
 'contra costa': 'contra costa',
 'wake': 'wake',
 'bergen': 'bergen',
 'westchester': 'westchester',
 'fort bend': 'fort bend',
 'chelan': 'chelan',
 'jefferson': 'jefferson',
 'santa rosa': 'santa r

In [94]:
ddf = ddf.assign(
    join_county=ddf.processed_county.map(str_map_4)
)

In [97]:
ddf_zip_map

,zip,type,primary_city,acceptable_cities,unacceptable_cities,state,county,timezone,area_codes,latitude,longitude,world_region,country,decommissioned,estimated_population,notes,processed_county
npartitions=12,,,,,,,,,,,,,,,,,
0,int64,object,object,object,object,object,object,object,object,float64,float64,object,object,int64,int64,object,object
3544,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38984,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42521,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [104]:
# ddf_final = pd.merge(
#     ddf,
#     ddf_zip_map[['zip', 'processed_county']],
#     how='left',
#     left_on='join_county',
#     right_on='processed_county'
# ).compute()
import partd

ddf_final = ddf.merge(
    ddf_zip_map[['zip', 'processed_county']], 
    how='left', 
    left_on='join_county', 
    right_on='processed_county'
).compute()


In [105]:
ddf_final

,date,county,state,fips,cases,deaths,processed_county_x,join_county,zip,processed_county_y
0,2020-04-03,Salt Lake,Utah,49035.0,541,1,salt lake,salt lake,84006,salt lake
1,2020-04-03,Salt Lake,Utah,49035.0,541,1,salt lake,salt lake,84020,salt lake
2,2020-04-03,Salt Lake,Utah,49035.0,541,1,salt lake,salt lake,84044,salt lake
3,2020-04-03,Salt Lake,Utah,49035.0,541,1,salt lake,salt lake,84047,salt lake
4,2020-04-03,Salt Lake,Utah,49035.0,541,1,salt lake,salt lake,84065,salt lake
...,...,...,...,...,...,...,...,...,...,...
135060,2020-04-06,Brown,Ohio,39015.0,3,1,brown,brown,76804,brown
135061,2020-04-06,Brown,Ohio,39015.0,3,1,brown,brown,76823,brown
135062,2020-04-06,Brown,Ohio,39015.0,3,1,brown,brown,76827,brown
135063,2020-04-06,Brown,Ohio,39015.0,3,1,brown,brown,76857,brown


# Full implementation

In [325]:
def preprocess_county(county_col):
    """Lower and remove County from end"""
    return county_col.str.lower().str.replace(' county', '')

def preprocess_state(state_col):
    df_state_map = pd.read_csv(
        'https://worldpopulationreview.com/static/states/abbr-name.csv',
        header=None
    ).rename(columns={0: 'abbreviation', 1: 'full'})
    df_state_map = df_state_map[['full', 'abbreviation']]

    df_state_map = df_state_map.assign(
        full=df_state_map.full.str.lower(),
        abbreviation=df_state_map.abbreviation.str.lower()
    )
    
    df_state_dict_map = dict(sorted(df_state_map.values.tolist(), reverse=True))
    state_col = state_col.str.lower().map(df_state_dict_map)

    return state_col


def lower(col):
    return col.str.lower()


def levenshtein_distance(str1, str2):
    return Levenshtein.distance(str1, str2)

In [ ]:
def fuzzy_join(
    left,
    right,
    left_on=(None,),
    right_on=(None,),
    left_preprocess_funcs=(None,),
    right_preprocess_funcs=(None,),
    **kwargs
):
    """
    :param left:
    :param right:
    :param left_on:
    :param right_on:
    :param implementation: In list ['serial', 'multiprocessing', 'dask']
    :return:
    """
    
    # TODO: Add dask (i.e. instantiate dask dataframes)
    
    
    
    # Preprocess each of the join columns
    left = left.assign(**{f'preprocessed_{left_on}': left_preprocess_func(left[left_on])})
    right = right.assign(**{f'preprocessed_{right_on}': right_preprocess_func(right[right_on])})
    
    # Get unique values preprocessed join columns
    left_unique = left[f'preprocessed_{left_on}'].dropna().unique()
    right_unique = right[f'preprocessed_{right_on}'].dropna().unique()

    # Calculate map
    fuzzy_map = fuzzy_match(left_unique, right_unique, **kwargs)
    
    # Apply map to create join column
    left = left.assign(**{f'join_{left_on}': left[f'preprocessed_{left_on}'].map(fuzzy_map)})
    
    # Rename preprocessed col to join col for clarity
    right = right.rename(columns={f'preprocessed_{right_on}': f'join_{right_on}'})
    
    # Perform Join
    df_final = pd.merge(
        left,
        right[['zip', f'join_{right_on}']],
        how='left',
        left_on=f'join_{left_on}',
        right_on=f'join_{right_on}'
    )


    return df_final

In [389]:
def fuzzy_join(
    left,
    right,
    left_on=[],
    right_on=[],
    **kwargs
):
    """
    :param left:
    :param right:
    :param left_on:
    :param right_on:
    :param implementation: In list ['serial', 'multiprocessing', 'dask']
    :return:
    """
    
    # TODO: Add dask (i.e. instantiate dask dataframes)
    
    assert len(left_on) == len(right_on)
    
    # Preprocess each of the join columns, create joint join column
    
    left_joint_join_col_name = '_'.join([config['col'] for config in left_on])
    preprocessed_left_joint_join_col_name = 'preprocessed_' + left_joint_join_col_name
    left = left.assign(**{preprocessed_left_joint_join_col_name: ''})
    for col_config in left_on:
        preprocessed_col = col_config['preprocess_func'](left[col_config["col"]])
        left = left.assign(
            **{preprocessed_left_joint_join_col_name: left[preprocessed_left_joint_join_col_name] + preprocessed_col + '_'}
        )
        
    right_joint_join_col_name = '_'.join([config['col'] for config in right_on])
    preprocessed_right_joint_join_col_name = 'preprocessed_' + right_joint_join_col_name
    right = right.assign(**{preprocessed_right_joint_join_col_name: ''})
    for col_config in right_on:
        preprocessed_col = col_config['preprocess_func'](right[col_config["col"]])
        right = right.assign(
            **{preprocessed_right_joint_join_col_name: right[preprocessed_right_joint_join_col_name] + preprocessed_col + '_'}
        )
    
    # Get unique values preprocessed join columns
    left_unique = left[preprocessed_left_joint_join_col_name].dropna().unique()
    right_unique = right[preprocessed_right_joint_join_col_name].dropna().unique()
    
    # Calculate map
    fuzzy_map = fuzzy_match(left_unique, right_unique, **kwargs)
    
    # Apply map to create join column
    left = left.assign(
        **{f'join_{left_joint_join_col_name}': left[preprocessed_left_joint_join_col_name].map(fuzzy_map)}
    )
    
    # Rename preprocessed col to join col for clarity
    right = right.rename(columns={preprocessed_right_joint_join_col_name: f'join_{right_joint_join_col_name}'})

    # Perform Join
    df_merged = pd.merge(
        left,
        right,
        how='left',
        left_on=f'join_{left_joint_join_col_name}',
        right_on=f'join_{right_joint_join_col_name}'
    )

    return df_merged

In [414]:
l, r, df_final = fuzzy_join(
    df,
    df_zip_map,
    left_on=[
        {'col': 'county', 'preprocess_func': preprocess_county},
        {'col': 'state', 'preprocess_func': preprocess_state}
    ],
    right_on=[
        {'col': 'county', 'preprocess_func': preprocess_county},
        {'col': 'state', 'preprocess_func': lower}
    ],
    fuzzy_distance_func=levenshtein_distance,
    implementation='dask',
    num_workers=10
)

hit 10


In [263]:
import multiprocessing
from itertools import repeat
from dask import delayed

def fuzzy_match(
    map_from,
    map_to,
    fuzzy_distance_func=levenshtein_distance, 
    implementation='serial',
    num_workers=None
):
    
    map_from_arr = pd.Series(map_from).dropna().unique()
    map_to_arr = pd.Series(map_to).dropna().unique()
    
    fuzzy_distance_func_vectorized = np.vectorize(fuzzy_distance_func)
    
    if implementation == 'serial':
        fuzzy_map = {}
        for x in map_from_arr:
            distances = fuzzy_distance_func_vectorized(x, map_to_arr)
            min_idx = np.argmin(distances)
            fuzzy_map[x] = map_to_arr[min_idx]
    
    elif implementation == 'multiprocessing':
        nproc = multiprocessing.cpu_count()
        map_from_arr_split = np.array_split(map_from_arr, nproc)
        
        with multiprocessing.Pool(nproc) as pool:
            results = pool.starmap(
                fuzzy_match_chunked, 
                zip(map_from_arr_split, repeat(map_to_arr), repeat(fuzzy_distance_func_vectorized))
            )
            
        fuzzy_map = {k: v for d in results for k, v in d.items()}
        
    elif implementation == 'dask':
        fuzzy_match_chunked_dask = delayed(fuzzy_match_chunked)

        if num_workers is None:
            num_workers = multiprocessing.cpu_count()
            
        results = []
        for map_from_arr_chunk in np.array_split(map_from_arr, num_workers):
            results.append(
                fuzzy_match_chunked_dask(map_from_arr_chunk, map_to_arr, fuzzy_distance_func_vectorized)
            )
        
        results = delayed(results).compute()
        fuzzy_map = {k: v for d in results for k, v in d.items()}
        
    return fuzzy_map
  

def fuzzy_match_chunked(map_from_arr_chunk, map_to_arr, distance_func):
    fuzzy_map = {}
    for x in map_from_arr_chunk:
        distances = distance_func(x, map_to_arr)
        min_idx = np.argmin(distances)
        fuzzy_map[x] = map_to_arr[min_idx]
    return fuzzy_map

In [268]:
%%time
res = fuzzy_match(
    preprocess_county(df.county.dropna()).unique(), 
    preprocess_county(df_zip_map.county.dropna()).unique(),
    implementation='dask',
    num_workers=12
)

hit 12
CPU times: user 1.49 s, sys: 25.8 ms, total: 1.51 s
Wall time: 1.5 s


In [269]:
res

{'snohomish': 'snohomish',
 'cook': 'cook',
 'orange': 'orange',
 'maricopa': 'maricopa',
 'los angeles': 'los angeles',
 'santa clara': 'santa clara',
 'suffolk': 'suffolk',
 'san francisco': 'san francisco',
 'dane': 'dane',
 'san diego': 'san diego',
 'bexar': 'bexar',
 'douglas': 'douglas',
 'humboldt': 'humboldt',
 'sacramento': 'sacramento',
 'solano': 'solano',
 'spokane': 'spokane',
 'salt lake': 'salt lake',
 'marin': 'marin',
 'napa': 'napa',
 'sonoma': 'sonoma',
 'washington': 'washington',
 'king': 'king',
 'alameda': 'alameda',
 'hillsborough': 'hillsborough',
 'manatee': 'manatee',
 'new york city': 'new york',
 'unknown': 'union',
 'placer': 'placer',
 'san mateo': 'san mateo',
 'fulton': 'fulton',
 'norfolk': 'norfolk',
 'grafton': 'grafton',
 'walla walla': 'walla walla',
 'contra costa': 'contra costa',
 'wake': 'wake',
 'bergen': 'bergen',
 'westchester': 'westchester',
 'fort bend': 'fort bend',
 'chelan': 'chelan',
 'jefferson': 'jefferson',
 'santa rosa': 'santa r

### References
* https://stackoverflow.com/questions/45545110/how-do-you-parallelize-apply-on-pandas-dataframes-making-use-of-all-cores-on-o
* https://stackoverflow.com/questions/5442910/python-multiprocessing-pool-map-for-multiple-arguments
* https://stackoverflow.com/questions/26784164/pandas-multiprocessing-apply
* https://stackoverflow.com/questions/25510482/python-pandas-multiprocessing-apply
* https://docs.dask.org/en/latest/dataframe-api.html#dask.dataframe.DataFrame.map_partitions

### Ideas for combination of distance metrics
 * https://pypi.org/project/python-Levenshtein/0.12.0/
 * https://rawgit.com/ztane/python-Levenshtein/master/docs/Levenshtein.html#Levenshtein-distance
 * https://towardsdatascience.com/fuzzy-string-matching-in-python-68f240d910fe

In [186]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [264]:
dist1 = Levenshtein.distance('Cook', 'Cook County')
ratio_1 = fuzz.ratio('Cook', 'Cook County')
print('distance: ', dist1, ' ratio: ', ratio_1)
print('Similarity score: ', ratio_1 / dist1)

distance:  7  ratio:  53
Similarity score:  7.571428571428571


In [265]:
dist2 = Levenshtein.distance('Cook', 'Carolina')
ratio_2 = fuzz.ratio('Cook', 'Carolina')
print('distance: ', dist2, ' ratio: ', ratio_2)
print('Similarity score: ', ratio_2 / dist2)


distance:  6  ratio:  33
Similarity score:  5.5


In [266]:
dist2 = Levenshtein.distance('Cook', 'Cooke')
ratio_2 = fuzz.ratio('Cook', 'Cooke')
print('distance: ', dist2, ' ratio: ', ratio_2)
print('Similarity score: ', ratio_2 / dist2)


distance:  1  ratio:  89
Similarity score:  89.0


# Try import

In [417]:
sys.path.append('/Users/nathanieldake/development/unsupervised/notebooks/utils/join_chain')


In [418]:
import jc_support

In [419]:
df.head(1)

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0


In [420]:
df_zip_map.head(1)

,zip,type,primary_city,acceptable_cities,unacceptable_cities,state,county,timezone,area_codes,latitude,longitude,world_region,country,decommissioned,estimated_population,notes
0,501,UNIQUE,Holtsville,NaN,I R S Service Center,NY,Suffolk County,America/New_York,631,40.81,-73.04,NaN,US,0,384,NaN


In [431]:
df_merged = jc_support.JoinChainSupport.fuzzy_join(
    df,
    df_zip_map,
    left_on=[
        {'col': 'county', 'preprocess_func': preprocess_county},
        {'col': 'state', 'preprocess_func': preprocess_state}
    ],
    right_on=[
        {'col': 'county', 'preprocess_func': preprocess_county},
        {'col': 'state', 'preprocess_func': lower}
    ],
    fuzzy_distance_func=levenshtein_distance,
    implementation='dask',
    num_workers=10
)

In [432]:
df_merged.shape

(943323, 24)

In [433]:
df_merged.head()

,date,county_x,state_x,fips,cases,deaths,preprocessed_county_state,join_county_state,zip,type,...,county_y,timezone,area_codes,latitude,longitude,world_region,country,decommissioned,estimated_population,notes
0,2020-01-21,Snohomish,Washington,53061.0,1,0,snohomish_wa_,snohomish_wa_,98012,STANDARD,...,Snohomish County,America/Los_Angeles,"425,206",47.84,-122.19,NaN,US,0,45281,NaN
1,2020-01-21,Snohomish,Washington,53061.0,1,0,snohomish_wa_,snohomish_wa_,98020,STANDARD,...,Snohomish County,America/Los_Angeles,"206,425",47.80,-122.37,NaN,US,0,17301,NaN
2,2020-01-21,Snohomish,Washington,53061.0,1,0,snohomish_wa_,snohomish_wa_,98021,STANDARD,...,Snohomish County,America/Los_Angeles,"425,206",47.79,-122.20,NaN,US,0,22499,NaN
3,2020-01-21,Snohomish,Washington,53061.0,1,0,snohomish_wa_,snohomish_wa_,98026,STANDARD,...,Snohomish County,America/Los_Angeles,"425,206",47.83,-122.32,NaN,US,0,30510,NaN
4,2020-01-21,Snohomish,Washington,53061.0,1,0,snohomish_wa_,snohomish_wa_,98036,STANDARD,...,Snohomish County,America/Los_Angeles,"425,360,206",47.80,-122.28,NaN,US,0,29598,NaN


In [430]:
def preprocess_county(county_col):
    """Lower and remove County from end"""
    return county_col.str.lower().str.replace(' county', '')

def preprocess_state(state_col):
    df_state_map = pd.read_csv(
        'https://worldpopulationreview.com/static/states/abbr-name.csv',
        header=None
    ).rename(columns={0: 'abbreviation', 1: 'full'})
    df_state_map = df_state_map[['full', 'abbreviation']]

    df_state_map = df_state_map.assign(
        full=df_state_map.full.str.lower(),
        abbreviation=df_state_map.abbreviation.str.lower()
    )
    
    df_state_dict_map = dict(sorted(df_state_map.values.tolist(), reverse=True))
    state_col = state_col.str.lower().map(df_state_dict_map)

    return state_col


def lower(col):
    return col.str.lower()